In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Loading training And Testing Dataset.....
The given dataset is in json format

In [ ]:
x=pd.read_json('../input/train.json')
y=pd.read_json('../input/test.json')

Loading Various preprocessing element for Natural Language Processing.......

In [ ]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
import string
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
z=x['cuisine']
from sklearn.preprocessing import LabelEncoder as le
from sklearn.preprocessing import OneHotEncoder as ohe


describe train dataset

In [ ]:
x.info()

In [ ]:
x.head()

describe test dataset

In [ ]:
y.info()

In [ ]:
y.head()

Converting list of ingredients to a sentence where each ingredient is separated by a space

In [ ]:
x['separated_ing']=x['ingredients'].map(lambda x: ' '.join(x))
y['separated_ing']=y['ingredients'].map(lambda x: ' '.join(x))


Preprocessing the sentence formed off in the above steps to 
-coverting all characters of ingredients to smalll letter
-remove punctuation marks
-converting multiple spaces to single spaces
Both Test and Train Data preprocessed


In [ ]:
import string,re
def purify(f):
    f=f.lower()
    f=re.sub('[%s]' % re.escape(string.punctuation),'',f)
    f=re.sub('\s+',' ',f)
    return f
x['cleared_ing']=x['separated_ing'].map(lambda g :purify(g))
y['cleared_ing']=y['separated_ing'].map(lambda g :purify(g))

Using a SnowballStemmer with english as parameter representing English language and also WordNetLemmatizer for bringing all ingredients to core form .Like 'works' -->'work'
This has been used by both training and test dataset

In [ ]:
sb=SnowballStemmer('english')
def stemmer(f):
    lists=[sb.stem(c) for c in f.split(" ")]
    return lists
l=WordNetLemmatizer()
def lemmar(f):
    lists=[l.lemmatize(g) for g in f.split(" ")]
    return lists
x['separated_ing_stemmed']=[stemmer(l) for l in x['cleared_ing']]
x['separated_ing_stemmed']=x['separated_ing_stemmed'].map(lambda x: ' '.join(x))
x['separated_ing_lemma']=[lemmar(l) for l in x['separated_ing_stemmed']]
x['separated_ing_lemma']=x['separated_ing_lemma'].map(lambda x: ' '.join(x))
y['separated_ing_stemmed']=[stemmer(l) for l in y['cleared_ing']]
y['separated_ing_stemmed']=y['separated_ing_stemmed'].map(lambda x: ' '.join(x))
y['separated_ing_lemma']=[lemmar(l) for l in y['separated_ing_stemmed']]
y['separated_ing_lemma']=y['separated_ing_lemma'].map(lambda x: ' '.join(x))

Dropping all unnecessary data columns from train and test dataset

In [ ]:
x=x.drop(['ingredients','separated_ing','cleared_ing','separated_ing_stemmed'],axis=1)
y=y.drop(['ingredients','separated_ing','cleared_ing','separated_ing_stemmed'],axis=1)

In [ ]:
x.columns

In [ ]:
a=''
def f(x):
    global a
    a+=x
for c,d in x.iterrows():
    f(d['separated_ing_lemma'])

In [ ]:
a=[x for x in a.split(" ")]
from sklearn.preprocessing import LabelEncoder as le
c=le().fit_transform(a)
c

In [ ]:
p=[]
for x1 in x['cuisine']:
    if x1 not in p:
        p.append(x1)
dic={}
for x1 in p:
    dic[x1]=[]
def gg(x):
    dic[x['cuisine']]=list(dic[x['cuisine']])+[f for f in x['separated_ing_lemma'].split(" ") if f not in dic[x['cuisine']]]
for e,d in x.iterrows():   
    gg(d)
    
    

In [ ]:
xx=pd.DataFrame(index=p,columns=p)

In [ ]:
b=[]
for n in a:
    if n not in b:
        b.append(n)
        

In [ ]:
xx1=xx

In [ ]:
for z in p:
    for z1 in p:
        xx.loc[z,z1]=len(list(set(dic[z]) & set(dic[z1])))/len(list(set(dic[z]) | set(dic[z1])))
        xx.loc[z1,z]=xx.loc[z,z1]

In [ ]:
xx.to_csv("cor.csv")

Using STOP_WORDS to remove all common ingredients present in the recipes 

lists=list(ENGLISH_STOP_WORDS)+stopwords.words()

Importing TFIDFVectorizer-->Term Frequency*Inverse Document frequency (Count of an element in doc*log(no. of doc/docs in which element present)) and CountVectorizer(count of each element in a row as vector)

from sklearn.feature_extraction.text import TfidfVectorizer  as tfidf,CountVectorizer as cv

Target as series z

z=x['cuisine']

setting parameters for itidf 
-max_df relates to maximum doc freq to be cosidered
-stop_words eliminate common words from given text
-analyzer how the text to be analyzed,word by word or character by character
fitting tf-idf over train dataset

tfidf1=tfidf(max_df=0.9,stop_words=lists,analyzer=u'word')
train=tfidf1.fit_transform(x['separated_ing_lemma'])
test=tfidf1.transform(y['separated_ing_lemma'])

Importing required ML functions from sklearn

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV as gsc
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier as xgb
from lightgbm import LGBMClassifier as lgb
from sklearn.linear_model import LogisticRegression as lr

Setting required parameters for svm

svm={'C':[6]}


Fitting  labelencoder over target to convert categorical target to numeric in nature 

from sklearn.preprocessing import LabelEncoder as le
p=le().fit(z)

z=p.transform(z)

Performing validation train  split of train dataset in ratio 3:7

from sklearn.model_selection import train_test_split as tts
xtrain,xtest,ztrain,ztest=tts(train,z,train_size=0.7)

Setting  LGBM parameters

r1=lgb(n_estimators=500,max_depth=7,objective='multiclass',metric='multi_logloss',num_classes=20,bagging_fraction=0.6,feature_fraction=0.6)

from sklearn.model_selection import GridSearchCV as gsc
a=gsc(lr(),svm)

Checking KNearestNeighbors over the data

from sklearn.neighbors import KNeighborsClassifier as knn
k={'n_neighbors':[5,7,9]}
k1=gsc(knn(),k)

Using Voting Classifier for classification purpose with
-SVM
-KNN
-LGBM
using soft margin for classification


from sklearn.ensemble import VotingClassifier as vc


v=vc(estimators=[('lr',a),('k1',k1),('lg',r1)],voting='soft')

Fitting train data over votingclassifier

v.fit(xtrain,ztrain)

Checking accuracy_score

from sklearn.metrics import accuracy_score 
print(accuracy_score(ztest,v.predict(xtest)))

Predicting test data target

z1=v.predict(test)

converting  numeric target back to categorical data

z=p.inverse_transform(z1)

Preparing output result file

ff=pd.DataFrame(z,index=y['id'],columns=['cuisine'])

ff.index.name='id'

ff.to_csv('aagya.csv')